In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import load
import pathlib
import pytz

2024-01-09 09:07:01.477591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nwb_starting_time_step = datetime(year=2022, month=1, day=1)
now_time = pytz.utc.localize(datetime(year=2024, month=1, day=1))

# now_time = pytz.utc.localize(datetime.now())
# nwb_starting_time_step = datetime.now() - relativedelta(days=360)
# nwb_starting_time_step = nwb_starting_time_step.strftime('%Y-%m-%d %H:%M:%S')
# nwb_starting_time_step = datetime.strptime(nwb_starting_time_step, "%Y-%m-%d %H:%M:%S")

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True,
    "temp_data_date": "2024-01-02"
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
stck_buffs = nwb_dfs[2]
pd.set_option('display.max_rows', None)
temp_df = stck_buffs[stck_buffs["index"] == 29][['captured_at', 'price_snapshot', 'x_day_ma']]
temp_df = temp_df[temp_df['captured_at'] <= '2023-06-27'].tail(20)
display(temp_df)
middle_column_sum = temp_df['price_snapshot'].sum()
print("middle_column_sum", middle_column_sum/20)
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_market.reset()

total_init_potfolio = nwb_market.wallet_state
for initd in range(100):
    data_initd = initd+1
    total_init_potfolio = total_init_potfolio + nwb_market.stock_data[data_initd]["price_snapshot"]*nwb_market.shares_data[data_initd]
print(total_init_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))

nwb_forest = Forester(market=nwb_market)
total_potfolio = nwb_market.wallet_state
latest_action = None
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]

initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step, now_time)
for indx in range(550):
    if indx < 20:
        acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        rand_acts = nwb_forest.market.get_mov_avg_actions()
        # print(rand_acts)
        acts_list = []
        iny = 1
        # rand_acts = np.full(100, 15)
        # if nwb_market.is_new_month:
        #     acts = np.full(100, 15)
        #     rand_acts = nwb_forest.act_random(acts)
        # else:
        #     rand_acts = nwb_forest.act_random(rand_acts, use_best_stks=False)
        rand_acts = nwb_forest.act_random(rand_acts, use_best_stks=False)
        for an_act in rand_acts:
            acts_list.append((iny, an_act))
            iny=iny+1
        print(nwb_market.the_current_time_step, nwb_market.the_current_time_step.day, now_time.day)
        # print(acts_list)
        nwb_market.step(rand_acts)
        print("abs_change_in_shares", nwb_market.abs_change_in_shares)
        if nwb_market.the_current_time_step > now_time :
            latest_action = acts
            break
        print("")

print(nwb_market.wallet_state)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:392: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:393: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:422: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

,captured_at,price_snapshot,x_day_ma
12614,2023-05-31 00:00:00+00:00,95.26,99.585000
12615,2023-06-01 00:00:00+00:00,95.85,99.290000
12616,2023-06-02 00:00:00+00:00,95.51,98.839286
12617,2023-06-05 00:00:00+00:00,94.91,98.237857
12618,2023-06-06 00:00:00+00:00,96.13,97.797857
12619,2023-06-07 00:00:00+00:00,101.31,97.889286
12620,2023-06-08 00:00:00+00:00,102.96,98.012857
12621,2023-06-09 00:00:00+00:00,104.86,98.315714
12622,2023-06-12 00:00:00+00:00,105.59,98.667857
12623,2023-06-13 00:00:00+00:00,105.85,99.040000


middle_column_sum 95.808
271740.7899999999
2021-12-30 20:00:00+00:00
271740.7899999999
2022-01-04 00:00:00+00:00 2024-01-01 00:00:00+00:00
stepping_b: 2022-01-04 00:00:00+00:00 274540.12999999995, 275740.7899999999, 275740.78999999986 0
total_sum 104000.0
stepping_b: 2022-01-05 00:00:00+00:00 280421.38133280707, 280177.65106925886, 280177.6510692589 0.0020795295121048348
total_sum 4000.0
stepping_b: 2022-01-06 00:00:00+00:00 284177.65106925886, 282338.12731519766, 282338.1273151977 -0.009960031122130926
total_sum 3999.9999999999955
stepping_b: 2022-01-07 00:00:00+00:00 286338.12731519766, 288987.34088431613, 288987.34088431613 0.008323382477034971
total_sum 3999.9999999999955
stepping_b: 2022-01-10 00:00:00+00:00 292570.5950794684, 295276.6998817548, 295276.6998817548 0.009456153165190304
total_sum 4000.000000000001
stepping_b: 2022-01-11 00:00:00+00:00 299276.6998817549, 299918.29751030565, 299918.29751030565 0.0021278903946311753
total_sum 4000.000000000001
stepping_b: 2022-01-12 00:

In [3]:
print(nwb_market.shares_data)
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
print(nwb_market.total_added)
gain = final_total_potfolio - initial_total_potfolio - nwb_market.total_added
print("gain", gain)

{1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 0.0, 48: 0.0, 49: 0.0, 50: 0.0, 51: 0.0, 52: 0.0, 53: 0.0, 54: 0.0, 55: 0.0, 56: 0.0, 57: 0.0, 58: 0.0, 59: 0.0, 60: 0.0, 61: 0.0, 62: 0.0, 63: 0.0, 64: 0.0, 65: 0.0, 66: 0.0, 67: 0.0, 68: 0.0, 69: 0.0, 70: 0.0, 71: 0.0, 72: 0.0, 73: 0.0, 74: 0.0, 75: 0.0, 76: 0.0, 77: 0.0, 78: 0.0, 79: 0.0, 80: 0.0, 81: 0.0, 82: 0.0, 83: 0.0, 84: 0.0, 85: 0.0, 86: 0.0, 87: 0.0, 88: 0.0, 89: 0.0, 90: 0.0, 91: 0.0, 92: 0.0, 93: 0.0, 94: 0.0, 95: 0.0, 96: 0.0, 97: 0.0, 98: 0.0, 99: 0.0, 100: 0.0}
3181084.20131505
2023-12-28 20:00:00+00:00
2080000
gain 829343.41131505


In [ ]:
import numpy as np
# 20035.532210277743
# Assuming avg_acts is your array and max_val, min_val are constants
# Replace them with your actual values

avg_acts = np.array([1, -2, 3, -4, 5])
max_val = 10
min_val = -10

# Separate positive and negative values
positive_values = avg_acts[avg_acts >= 0]
negative_values = avg_acts[avg_acts < 0]

# Calculate the scaled array for positive values
scaled_positive = positive_values * ((max_val - 0) / (positive_values.max() - 0))

# Calculate the scaled array for negative values
scaled_negative = negative_values * ((0 - min_val) / (0 - negative_values.min()))

# Combine the scaled arrays back together
scaled_array = np.empty_like(avg_acts)
scaled_array[avg_acts >= 0] = scaled_positive
scaled_array[avg_acts < 0] = scaled_negative

print(scaled_array)